In [1]:
%matplotlib inline

import random
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import torch
import dense_correspondence_manipulation.utils.utils as utils
utils.add_dense_correspondence_to_python_path()

import dense_correspondence
from dense_correspondence.evaluation.evaluation import *
import dense_correspondence.correspondence_tools.correspondence_plotter as correspondence_plotter
from dense_correspondence.dataset.dense_correspondence_dataset_masked import ImageType

/home/peteflo/code/modules/dense_correspondence_manipulation/utils/transformations.py:1912: UserWarning: failed to import module _transformations
  warnings.warn('failed to import module %s' % name)


In [2]:
config_filename = os.path.join(utils.getDenseCorrespondenceSourceDir(), 'config', 
                               'dense_correspondence', 'evaluation', 'evaluation.yaml')
config = utils.getDictFromYamlFilename(config_filename)
default_config = utils.get_defaults_config()


utils.set_cuda_visible_devices([0])
dce = DenseCorrespondenceEvaluation(config)
DCE = DenseCorrespondenceEvaluation


network_name = "3_caterpillar_m_background_1.0"
dcn = dce.load_network_from_config(network_name)
dcn.eval()
dataset = dcn.load_training_dataset()

setting CUDA_VISIBLE_DEVICES =  0,


/usr/local/lib/python2.7/dist-packages/torch/nn/modules/module.py:482: UserWarning: src is not broadcastable to dst, but they have the same number of elements.  Falling back to deprecated pointwise behavior.
  own_state[name].copy_(param)


Using SpartanDataset:
   - in train mode
   - number of scenes 17
   - total images:     5242


In [11]:
from PIL import Image
from plotting import normalize_descriptor
import time

## Usage:
## set the network above
## set the full_rgb_only_path in last cell of notebook (if you'd like can use log_list too)
## run the full notebook

descriptor_image_stats = dcn.descriptor_image_stats

def make_descriptors_images(full_rgb_only_path):
    for i in sorted(os.listdir(full_rgb_only_path)):
        #print i
        start = time.time()
        if "rgb.png" not in i:
            continue

        rgb_filename = os.path.join(full_rgb_only_path, i)
        rgb_a = Image.open(rgb_filename).convert('RGB')

        # compute dense descriptors
        # This takes in a PIL image!
        rgb_a_tensor = dataset.rgb_image_to_tensor(rgb_a)

        # these are Variables holding torch.FloatTensors, first grab the data, then convert to numpy
        res_a = dcn.forward_single_image_tensor(rgb_a_tensor).data.cpu().numpy()
        res_a = normalize_descriptor(res_a, descriptor_image_stats["mask_image"])

#         # This chunk of code would produce masked descriptors
#         # MASK
#         mask_name = i.split("_rgb")[0] + "_mask.png"
#         mask_filename = os.path.join(os.path.dirname(rgb_filename), "../image_masks", mask_name)
#         mask = np.asarray(Image.open(mask_filename))
#         mask_three_channel = np.zeros((480,640,3))
#         for j in range(3):
#             mask_three_channel[:,:,j] = mask
#         res_a = res_a * mask_three_channel

        save_file_name = os.path.join(save_path, i)
    #     plt.imshow(res_a)
    #     plt.show()
    #     break
        plt.imsave(save_file_name, res_a)
        print "forward and saving at rate", time.time()-start

In [12]:
def make_videos(full_rgb_only_path, log):
    # make an rgb only dir
    rgb_only_path = os.path.join(os.path.dirname(full_rgb_only_path),"rgb_only")
    os.system("mkdir -p "+ rgb_only_path)
    os.system("cp "+full_rgb_only_path+"/*rgb.png "+ rgb_only_path)
    
    # make descriptor video
    video_des = log+"_video_descriptors.mp4"
    os.system("cd "+save_path+" && ffmpeg -framerate 30 -pattern_type glob -i '*.png' -c:v libx264 -r 30 "+video_des)
    # make rgb video
    video_rgb = log+"_video_rgb.mp4"
    os.system("cd "+rgb_only_path+" && ffmpeg -framerate 30 -pattern_type glob -i '*.png' -c:v libx264 -r 30 "+video_rgb)
    
    # make mac friendly?
    processed_path = os.path.dirname(rgb_only_path)
    rgb_video = os.path.join(rgb_only_path, video_rgb)
    descriptors_video = os.path.join(save_path, video_des)
    
    cmd = "cd "+processed_path+" && ffmpeg \
      -i "+rgb_video + " -pix_fmt yuv420p "+rgb_video.split(".mp4")[0]+"_mac.mp4"
    print cmd
    os.system(cmd)
    cmd = "cd "+processed_path+" && ffmpeg \
      -i "+descriptors_video + " -pix_fmt yuv420p "+descriptors_video.split(".mp4")[0]+"_mac.mp4"
    os.system(cmd)
    
    
    # merge the videos!
    cmd = "cd "+processed_path+" && ffmpeg \
      -i "+rgb_video +" \
      -i "+descriptors_video +"\
      -filter_complex '[0:v]pad=iw*2:ih[int];[int][1:v]overlay=W/2:0[vid]' \
      -map [vid] \
      -c:v libx264 \
      -crf 23 \
      -preset veryfast \
      output_"+log+".mp4"

    print cmd
    os.system(cmd)

In [16]:
# log_list = ["2018-04-10-16-02-59",
#             "2018-04-10-16-14-41",
#             "2018-04-10-16-05-17",
#             "2018-04-10-16-06-26",
#             "2018-04-10-16-12-21",
#             "2018-04-10-16-13-37"]

# log_list = ['2018-04-27-19-39-51'] # caterpillar tricks

# log_list  = ['2018-05-18-16-00-28'] # hats in isolation

log_list = ['2018-05-18-16-04-03'] # many hats

for log in log_list:
    full_rgb_only_path = "/media/peteflo/3TBbackup/dense_correspondence/pdc/logs_special/"+log+"/processed/images"
    save_path = os.path.join(full_rgb_only_path, "../descriptors_only_masked")
    print save_path
    os.system("mkdir -p "+ save_path)
    make_descriptors_images(full_rgb_only_path)
    make_videos(full_rgb_only_path, log)

/media/peteflo/3TBbackup/dense_correspondence/pdc/logs_special/2018-05-18-16-04-03/processed/images/../descriptors_only_masked
forward and saving at rate 0.166264057159
forward and saving at rate 0.130218982697
forward and saving at rate 0.127439022064
forward and saving at rate 0.118808984756
forward and saving at rate 0.112361907959
forward and saving at rate 0.1189930439
forward and saving at rate 0.112367868423
forward and saving at rate 0.123177051544
forward and saving at rate 0.113759994507
forward and saving at rate 0.116936922073
forward and saving at rate 0.112778902054
forward and saving at rate 0.117197990417
forward and saving at rate 0.108792066574
forward and saving at rate 0.108872890472
forward and saving at rate 0.107377052307
forward and saving at rate 0.111324071884
forward and saving at rate 0.104799985886
forward and saving at rate 0.107983827591
forward and saving at rate 0.108350992203
forward and saving at rate 0.141292095184
forward and saving at rate 0.116348

forward and saving at rate 0.166517972946
forward and saving at rate 0.165499925613
forward and saving at rate 0.164526939392
forward and saving at rate 0.163584947586
forward and saving at rate 0.165089130402
forward and saving at rate 0.164725780487
forward and saving at rate 0.171194076538
forward and saving at rate 0.167849063873
forward and saving at rate 0.167306900024
forward and saving at rate 0.164113998413
forward and saving at rate 0.163833141327
forward and saving at rate 0.162333965302
forward and saving at rate 0.16389298439
forward and saving at rate 0.163544893265
forward and saving at rate 0.165174007416
forward and saving at rate 0.162738084793
forward and saving at rate 0.164414167404
forward and saving at rate 0.166306018829
forward and saving at rate 0.165850162506
forward and saving at rate 0.17200088501
forward and saving at rate 0.165709018707
forward and saving at rate 0.163118124008
forward and saving at rate 0.164309978485
forward and saving at rate 0.1666288

forward and saving at rate 0.177706956863
forward and saving at rate 0.184571027756
forward and saving at rate 0.182775974274
forward and saving at rate 0.176517009735
forward and saving at rate 0.175493955612
forward and saving at rate 0.181063890457
forward and saving at rate 0.175297021866
forward and saving at rate 0.194453001022
forward and saving at rate 0.194126844406
forward and saving at rate 0.192355871201
forward and saving at rate 0.176553964615
forward and saving at rate 0.182109117508
forward and saving at rate 0.175928115845
forward and saving at rate 0.173002004623
forward and saving at rate 0.172672986984
forward and saving at rate 0.172378063202
forward and saving at rate 0.180999040604
forward and saving at rate 0.165203094482
forward and saving at rate 0.159607887268
forward and saving at rate 0.16136097908
forward and saving at rate 0.158344984055
forward and saving at rate 0.161580085754
forward and saving at rate 0.160923957825
forward and saving at rate 0.162710

forward and saving at rate 0.169945955276
forward and saving at rate 0.173621892929
forward and saving at rate 0.170674085617
forward and saving at rate 0.180276155472
forward and saving at rate 0.165943145752
forward and saving at rate 0.167397022247
forward and saving at rate 0.168209075928
forward and saving at rate 0.166090011597
forward and saving at rate 0.171020030975
forward and saving at rate 0.166053056717
forward and saving at rate 0.162785053253
forward and saving at rate 0.16210103035
forward and saving at rate 0.161996841431
forward and saving at rate 0.169008970261
forward and saving at rate 0.169944047928
forward and saving at rate 0.167619943619
forward and saving at rate 0.16495513916
forward and saving at rate 0.162386894226
forward and saving at rate 0.157719135284
forward and saving at rate 0.15505695343
forward and saving at rate 0.15634393692
forward and saving at rate 0.150845050812
forward and saving at rate 0.148661136627
forward and saving at rate 0.152054071

forward and saving at rate 0.167103052139
forward and saving at rate 0.162949085236
forward and saving at rate 0.164857149124
forward and saving at rate 0.166831970215
forward and saving at rate 0.164786100388
forward and saving at rate 0.175841808319
forward and saving at rate 0.165723085403
forward and saving at rate 0.167386054993
forward and saving at rate 0.164055109024
forward and saving at rate 0.166735887527
forward and saving at rate 0.166115999222
forward and saving at rate 0.162657976151
forward and saving at rate 0.165614843369
forward and saving at rate 0.164652109146
forward and saving at rate 0.164640903473
forward and saving at rate 0.166275024414
forward and saving at rate 0.165951967239
forward and saving at rate 0.171365022659
forward and saving at rate 0.163702011108
forward and saving at rate 0.164304018021
forward and saving at rate 0.172293186188
forward and saving at rate 0.167284011841
forward and saving at rate 0.166039943695
forward and saving at rate 0.17503

...

forward and saving at rate 0.154042005539
forward and saving at rate 0.153805017471
forward and saving at rate 0.152561903
forward and saving at rate 0.158725976944
forward and saving at rate 0.154479026794
forward and saving at rate 0.155999898911
forward and saving at rate 0.155055999756
forward and saving at rate 0.153645992279
forward and saving at rate 0.154371023178
forward and saving at rate 0.160420179367
forward and saving at rate 0.164615869522
forward and saving at rate 0.159698009491
forward and saving at rate 0.157121181488
forward and saving at rate 0.157835960388
forward and saving at rate 0.156175851822
forward and saving at rate 0.15549492836
forward and saving at rate 0.161906003952
forward and saving at rate 0.158766031265
forward and saving at rate 0.157595872879
forward and saving at rate 0.155554056168
forward and saving at rate 0.161118030548
forward and saving at rate 0.162423849106
forward and saving at rate 0.159161806107
forward and saving at rate 0.157963991

forward and saving at rate 0.14995598793
forward and saving at rate 0.150710105896
forward and saving at rate 0.14638209343
forward and saving at rate 0.148328781128
forward and saving at rate 0.148399829865
forward and saving at rate 0.148616075516
forward and saving at rate 0.149399995804
forward and saving at rate 0.153589010239
forward and saving at rate 0.152647018433
forward and saving at rate 0.154408931732
forward and saving at rate 0.155704021454
forward and saving at rate 0.154057979584
forward and saving at rate 0.158582925797
forward and saving at rate 0.154915094376
forward and saving at rate 0.158348083496
forward and saving at rate 0.156152009964
forward and saving at rate 0.151909828186
forward and saving at rate 0.153263092041
forward and saving at rate 0.152942895889
forward and saving at rate 0.206123113632
forward and saving at rate 0.170970916748
forward and saving at rate 0.170954942703
forward and saving at rate 0.172081947327
forward and saving at rate 0.1707289

forward and saving at rate 0.170166015625
forward and saving at rate 0.15309214592
forward and saving at rate 0.156692028046
forward and saving at rate 0.152575016022
forward and saving at rate 0.148682832718
forward and saving at rate 0.157809019089
forward and saving at rate 0.158401012421
forward and saving at rate 0.159176111221
forward and saving at rate 0.165529966354
forward and saving at rate 0.158807992935
forward and saving at rate 0.158487081528
forward and saving at rate 0.157520055771
forward and saving at rate 0.1590051651
forward and saving at rate 0.159841060638
forward and saving at rate 0.159624099731
forward and saving at rate 0.158953905106
forward and saving at rate 0.161287784576
forward and saving at rate 0.159552812576
forward and saving at rate 0.178347826004
forward and saving at rate 0.164644002914
forward and saving at rate 0.158493995667
forward and saving at rate 0.157689094543
forward and saving at rate 0.159186124802
forward and saving at rate 0.15846014

forward and saving at rate 0.159391880035
forward and saving at rate 0.158836126328
forward and saving at rate 0.160603046417
forward and saving at rate 0.15762090683
forward and saving at rate 0.160504102707
forward and saving at rate 0.176544904709
forward and saving at rate 0.165364980698
forward and saving at rate 0.160551071167
forward and saving at rate 0.161231994629
forward and saving at rate 0.159124851227
forward and saving at rate 0.15678191185
forward and saving at rate 0.165195941925
forward and saving at rate 0.160374879837
forward and saving at rate 0.1618039608
forward and saving at rate 0.165941953659
forward and saving at rate 0.159384965897
forward and saving at rate 0.159914970398
forward and saving at rate 0.160247087479
forward and saving at rate 0.162374019623
forward and saving at rate 0.175395965576
forward and saving at rate 0.158725976944
forward and saving at rate 0.159157991409
forward and saving at rate 0.163038015366
forward and saving at rate 0.159603118

forward and saving at rate 0.156581878662
forward and saving at rate 0.157924890518
forward and saving at rate 0.155710935593
forward and saving at rate 0.157539129257
forward and saving at rate 0.155306816101
forward and saving at rate 0.154628992081
forward and saving at rate 0.155972957611
forward and saving at rate 0.155257940292
forward and saving at rate 0.155779123306
forward and saving at rate 0.155505180359
forward and saving at rate 0.166750907898
forward and saving at rate 0.158434152603
forward and saving at rate 0.154989004135
forward and saving at rate 0.15557718277
forward and saving at rate 0.158107995987
forward and saving at rate 0.154644966125
forward and saving at rate 0.158478021622
forward and saving at rate 0.15522813797
forward and saving at rate 0.164026021957
forward and saving at rate 0.220183849335
forward and saving at rate 0.210066080093
forward and saving at rate 0.175882101059
forward and saving at rate 0.177197217941
forward and saving at rate 0.1578810

In [53]:
# sandboxing
#os.system("cd /media/peteflo/3TBbackup/dense_correspondence/pdc/logs_proto/2018-04-10-16-13-37/processed && ffmpeg -i /media/peteflo/3TBbackup/dense_correspondence/pdc/logs_proto/2018-04-10-16-13-37/processed/rgb_only/2018-04-10-16-13-37_video_rgb.mp4   -c:v libx264       -crf 23       -preset veryfast /media/peteflo/3TBbackup/dense_correspondence/pdc/logs_proto/2018-04-10-16-13-37/processed/rgb_only/2018-04-10-16-13-37_video_rgb_2.mp4")

256

In [44]:
# os.system("cd /media/peteflo/3TBbackup/dense_correspondence/pdc/logs_proto/2018-04-10-16-14-41/processed && ffmpeg       -i /media/peteflo/3TBbackup/dense_correspondence/pdc/logs_proto/2018-04-10-16-14-41/processed/rgb_only/2018-04-10-16-14-41_video_rgb.mp4 -pix_fmt yuv420p /media/peteflo/3TBbackup/dense_correspondence/pdc/logs_proto/2018-04-10-16-14-41/processed/rgb_only/2018-04-10-16-14-41_video_rgb_2.mp4")

0